In [ ]:
import spot
spot.setup()

# Formulae

## LTL3DRA fragment check
LTL3DRA works only for $\text{LTL}\setminus\mathsf{G}(\mathsf{UX})$

In [ ]:
def is_in_ltlgux(form):
    if type(form) == str:
        f = spot.formula(form)
    else:
        f = form
    res = !ltl3dra -C -f "{f.to_str(format='spin')}"
    if len(res) == 0:
        raise Exception("ltl3dra did not finished as expected")
    else:
        if res[0] == '0':
            return False
        elif res[0] == '1':
            return True
        else:
            raise Exception("ltl3dra did not finished as expected")

In this notebook we generate files with formulae that we further use for our evaluation. We use three types of formulae regarding their source. These are

1. **classical benchmarking formulae from literature**,
2. **parametric formulae from literature**, and
3. **random formulae**.

## Formulae from literature

* beem: R. Pelánek: _BEEM: Benchmarks for explicit model checkers._ Proceedings of Spin'07.  LNCS 4595.
* hkrss: Holeček, T. Kratochvila, V. Řehák, D. Šafránek, and P. Šimeček: _Verification  Results  in  Liberouter Project._  Tech. Report 03, CESNET, 2004.
* dac: M.  B.  Dwyer and G. S. Avrunin and J. C. Corbett: _Property Specification Patterns for Finite-state Verification._ Proceedings of FMSP'98.
* eh: K. Etessami and G. J. Holzmann: _Optimizing Büchi Automata._  Proceedings of Concur'00.  LNCS 1877.
* sb: F. Somenzi and R. Bloem: _Efficient Büchi Automata for LTL Formulae._  Proceedings of CAV'00.  LNCS 1855.

There are many duplicates among the sources mentioned above. Therefore, we process them to minimize the level of duplicates. The steps are:
1. The temporal operators $\mathsf{W}$ and $\mathsf{M}$ are removed by applying standard equivalences.
1. Atomic propositions are renamed in alphabetical order and also free boolean combinations of AP are replaced by fresh AP. For example, the formula $\mathsf{G}(a \lor b) \land \mathsf{G} c$ is relabelled to $\mathsf{G} a \land \mathsf{G} b$.
2. Formulae are simplified by SPOT's reductions techniques.
3. Formulae equivallent to `True` or `False` are removed.

Next, we add also a negations of already chosen formulae.

In [ ]:
tmp_file      = 'formulae/tmp.ltl'
lit_pref      = 'formulae/literature'
lit_file      = lit_pref + '.ltl'
rand_pref     = 'formulae/random'

In [ ]:
!genltl --dac-patterns --eh-patterns --sb-patterns --beem-patterns --hkrss-patterns > $tmp_file

In [ ]:
!ltlfilt --negate $tmp_file | \
ltlfilt $tmp_file -F - --unique -r3 --remove-wm --relabel-bool=abc | \
     ltlfilt -v --equivalent-to=0 | ltlfilt -v --equivalent-to=1> $lit_file

In [ ]:
count = sum(1 for line in open(lit_file))
print('Number of formulae from literature:\t{}'.format(count))

In [ ]:
!wc -l $lit_file

### Categorize into fragments

In [ ]:
def categorize(form_file,prefix):
    with open(form_file,'r') as source,\
         open('{}_ltl3dra.ltl'.format(prefix),'w') as ltl3dra,\
         open('{}_full.ltl'.format(prefix),'w') as full:
        for form in source:
            if is_in_ltlgux(form):
                print(form,file=ltl3dra,end='')
            else:
                print(form,file=full,end='')

In [ ]:
categorize(lit_file,lit_pref)

In [ ]:
def is_fg(f):
    if 'U' in f or 'R' in f or 'X' in f or 'M' in f or 'W' in f:
        return False
    else:
        return True

In [ ]:
for t in ['ltl3dra','full']:
    count = sum(1 for line in open('{}_{}.ltl'.format(lit_pref,t)))
    print('Formulae from LTL fragment {}:\t{}'.format(t,count))

In [ ]:
with open('{}_{}.ltl'.format(lit_pref,'ltl3dra')) as file:
    count = 0
    for line in file:
        if is_fg(line):
            count += 1
print('Formulae from LTL(F,G) that are included in the formulae of the fragment of ltl3dra:\t{} '.format(count))

## Parametric formulae

## Random formulae

In [ ]:
def generate(n=100,func=(lambda x: True),filename=None,seed=0,priorities='M=0,W=0,xor=0',ap=['a','b','c','d','e']):
    '''Generates a specified number of formulas that are
    accepted by the given function. If filename specified
    it prints them into the given file.
    '''
    if filename is None:
        file_h = sys.stdout
    else:
        file_h = open(filename,'w')
    f = spot.randltl(ap,ltl_priorities=priorities,
                  simplify=3,tree_size=15,seed=seed).\
                  relabel_bse(spot.Pnn).unabbreviate('WM')
    i = 0
    printed = set()
    c = spot.language_containment_checker()
    while(i < n):
        form = next(f)
        #form = spot.negative_normal_form(form)
        if form in printed:
            continue
        if func(form) and not c.equal(form, spot.formula.tt()) and not c.equal(form, spot.formula.ff()):
            print(form,file=file_h)
            printed.add(form)
            i += 1

generate 500 formulas for each fragment

In [ ]:
for t,f in [('ltl3dra',is_in_ltlgux),\
            ('full',(lambda x: not is_in_ltlgux(x)))]:
    generate(500,f,'{}_{}.ltl'.format(rand_pref,t))

In [ ]:
generate(500,filename='{}_fg.ltl'.format(rand_pref),priorities='xor=0,implies=0,equiv=0,X=0,W=0,M=0,R=0,U=0,F=3,G=3')

In [ ]:
for t in ['fg','ltl3dra','full']:
    count = sum(1 for line in open('{}_{}.ltl'.format(rand_pref,t)))
    print('Formulae from LTL fragment {}:\t{}'.format(t,count))